In [2]:
!pip install faker

     |████████████████████████████████| 1.9 MB 1.8 MB/s eta 0:00:01


In [3]:
from faker import Faker
import psycopg2
from tqdm import tqdm
import random

# Inicializa o Faker
fake = Faker('pt_BR')  # Locale brasileiro

# Conexão com o banco
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="admin",
    host="192.168.0.202",
    port="5432"
)
cursor = conn.cursor()

# Configurações
total_registros = 100_000
batch_size = 10_000  # Inserções em lote para performance
status_opcoes = ['ativo', 'inativo', 'pendente']

def gerar_dados_fake(inicio, qtd):
    dados = []
    for i in range(inicio, inicio + qtd):
        nome = fake.name()
        email = f"{nome.lower().replace(' ', '.')}.{i}@exemplo.com"
        data_cadastro = fake.date_between(start_date='-2y', end_date='today')
        status = random.choice(status_opcoes)
        dados.append((nome, email, data_cadastro, status))
    return dados

# Inserção em lotes
for inicio in tqdm(range(0, total_registros, batch_size)):
    dados = gerar_dados_fake(inicio, batch_size)
    args_str = ','.join(cursor.mogrify("(%s,%s,%s,%s)", x).decode("utf-8") for x in dados)
    cursor.execute(f"INSERT INTO clientes (nome, email, data_cadastro, status) VALUES {args_str}")
    conn.commit()

cursor.close()
conn.close()


100%|██████████| 10/10 [00:39<00:00,  3.92s/it]
